In [52]:
import pandas as pd
import numpy as np
import psycopg2
import pyarrow.parquet as pq
import polars as pl
import duckdb


In [3]:
# Read policy table to know the Schema and select required columns. (Raw Data is preloaded using direct download from NFIP web)
policies = pq.ParquetFile(r"D:\Self projects\Flood Insurance SQL-Python\data\FimaNfipPolicies.parquet")
schema = policies.schema

print(schema)

required group field_id=-1 duckdb_schema {
  optional boolean field_id=-1 agricultureStructureIndicator;
  optional int32 field_id=-1 baseFloodElevation (Decimal(precision=8, scale=2));
  optional int32 field_id=-1 basementEnclosureCrawlspaceType (Int(bitWidth=16, isSigned=true));
  optional int32 field_id=-1 cancellationDateOfFloodPolicy (Date);
  optional binary field_id=-1 condominiumCoverageTypeCode (String);
  optional boolean field_id=-1 construction;
  optional int32 field_id=-1 crsClassCode (Int(bitWidth=16, isSigned=true));
  optional binary field_id=-1 buildingDeductibleCode (String);
  optional binary field_id=-1 contentsDeductibleCode (String);
  optional boolean field_id=-1 elevatedBuildingIndicator;
  optional binary field_id=-1 elevationCertificateIndicator (String);
  optional int32 field_id=-1 elevationDifference (Int(bitWidth=32, isSigned=true));
  optional int32 field_id=-1 federalPolicyFee (Int(bitWidth=16, isSigned=true));
  optional binary field_id=-1 ratedFloodZo

In [4]:

#  COLUMNS TO RETAIN 
selected_columns = [
    "id",
    "reportedZipCode",
    "propertyState",
    "reportedCity",
    "countyCode",
    "latitude",
    "longitude",
    "policyEffectiveDate",
    "policyTerminationDate",
    "propertyPurchaseDate",
    "totalBuildingInsuranceCoverage",
    "totalContentsInsuranceCoverage",
    "totalInsurancePremiumOfThePolicy",
    "buildingReplacementCost",
    "occupancyType",
    "construction",
    "numberOfFloorsInInsuredBuilding",
    "lowestFloorElevation",
    "elevatedBuildingIndicator",
    "floodproofedIndicator",
    "postFIRMConstructionIndicator",
    "policyTermIndicator",
    "rateMethod",
    "programTypeIndicator",
    "nfipRatedCommunityNumber"
]

#  LOAD & SAVE WITH duckdb
con = duckdb.connect("D:\Self projects\Flood Insurance SQL-Python\data\insurance.duckdb")

# con.sql("""
#         CREATE TABLE IF NOT EXISTS policies
#         as from read_parquet("D:\Self projects\Flood Insurance SQL-Python\data\FimaNfipPolicies.parquet")
#         """)



In [12]:
[print(i,',') for i in selected_columns];

id ,
reportedZipCode ,
propertyState ,
reportedCity ,
countyCode ,
latitude ,
longitude ,
policyEffectiveDate ,
policyTerminationDate ,
propertyPurchaseDate ,
totalBuildingInsuranceCoverage ,
totalContentsInsuranceCoverage ,
totalInsurancePremiumOfThePolicy ,
buildingReplacementCost ,
occupancyType ,
construction ,
numberOfFloorsInInsuredBuilding ,
lowestFloorElevation ,
elevatedBuildingIndicator ,
floodproofedIndicator ,
postFIRMConstructionIndicator ,
policyTermIndicator ,
rateMethod ,
programTypeIndicator ,
nfipRatedCommunityNumber ,


In [21]:
sel_policies = con.sql("""
                        select 
                    id ,
                    reportedZipCode ,
                    propertyState ,
                    reportedCity ,
                    countyCode ,
                    latitude ,
                    longitude ,
                    policyEffectiveDate ,
                    policyTerminationDate ,
                    propertyPurchaseDate ,
                    totalBuildingInsuranceCoverage ,
                    totalContentsInsuranceCoverage ,
                    totalInsurancePremiumOfThePolicy ,
                    buildingReplacementCost ,
                    occupancyType ,
                    construction ,
                    numberOfFloorsInInsuredBuilding ,
                    lowestFloorElevation ,
                    elevatedBuildingIndicator ,
                    floodproofedIndicator ,
                    postFIRMConstructionIndicator ,
                    policyTermIndicator ,
                    rateMethod ,
                    programTypeIndicator ,
                    nfipRatedCommunityNumber
                        
                    From policies
                       
                    where propertyState in ('Tx','TX','LA','FL','NY','CA')
                    """)

In [22]:
con.sql("""
        select DISTINCT propertyState 
        from sel_policies
        """).df()['propertyState'].values

array(['FL', 'TX', 'NY', 'LA', 'Tx', 'CA'], dtype=object)

In [53]:
states = ['TX','LA','FL','NY','CA']

In [ ]:

## Create a CSV file of the policies data per selected state with selected columns
for state in states:
  con.sql(f"""
          COPY (
            SELECT * FROM sel_policies
            where propertyState = '{state}'
          ) TO "nfip_policies_{state}.csv" (HEADER, DELIMITER ',');
          """)

In [54]:
#Load the data into postgresql

from dotenv import load_dotenv
import os

#Create secure connection
load_dotenv('credentials.env')

try:
    post_con = psycopg2.connect(host = os.getenv('PG_HOST'), 
                            dbname = os.getenv('PG_DB'), 
                            user = os.getenv('PG_USER'), 
                            password = os.getenv('PG_PASSWORD'))
except psycopg2.Error as e:
    print(e)

In [55]:
# Get the cursor to execute SQL queries using the connection made above
cur = post_con.cursor()

# set autocommit = True to not save every query manually
post_con.set_session(autocommit=True)

In [56]:
# Create a database & connect to the new database by closing the connection to default database

try:
    cur.execute("Create database NFIP_Policies")
except psycopg2.Error as e:
    print(e)
post_con.close()

# Connect to new database just created (Database name is read in all small case letters)
try:
    post_con = psycopg2.connect(host = os.getenv('PG_HOST'), 
                            dbname = 'nfip_policies', 
                            user = os.getenv('PG_USER'), 
                            password = os.getenv('PG_PASSWORD'))
except psycopg2.Error as e:
    print(e)

database "nfip_policies" already exists



In [ ]:
# Create a new cursor for new database
cur = post_con.cursor()
post_con.set_session(autocommit=True)

In [ ]:
# create policies table into database using a data model
for state in states:
    cur.execute(f""" CREATE TABLE policies_{state} 
                (id UUID PRIMARY KEY,
                reportedZipCode TEXT,
                propertyState TEXT,
                reportedCity TEXT,
                countyCode TEXT,
                latitude NUMERIC(9,1),
                longitude NUMERIC(9,1),
                policyEffectiveDate DATE,
                policyTerminationDate DATE,
                propertyPurchaseDate DATE,
                totalBuildingInsuranceCoverage INTEGER,
                totalContentsInsuranceCoverage INTEGER,
                totalInsurancePremiumOfThePolicy INTEGER,
                buildingReplacementCost BIGINT,
                occupancyType SMALLINT,
                construction BOOLEAN,
                numberOfFloorsInInsuredBuilding SMALLINT,
                lowestFloorElevation NUMERIC(8,2),
                elevatedBuildingIndicator BOOLEAN,
                floodproofedIndicator BOOLEAN,
                postFIRMConstructionIndicator BOOLEAN,
                policyTermIndicator SMALLINT,
                rateMethod TEXT,
                programTypeIndicator BOOLEAN,
                nfipRatedCommunityNumber TEXT
                );
                """)

In [ ]:
# Insert the policies data from the CSV file
state_policies = os.listdir('Data')
for file,state in zip(state_policies,sorted(states)):
    with open(f"Data\{file}", "r") as f:
        cur.copy_expert(f"""
            COPY policies_{state} FROM STDIN WITH CSV HEADER
        """, f)


In [ ]:
# cur.execute("select * from policies_ca")

In [45]:
# Check if data is loaded to the table 
## cur.fetchone() OR cur.fetchall() OR
pd.read_sql("SELECT * FROM policies_ca LIMIT 10;", post_con)

C:\Users\pranj\AppData\Local\Temp\ipykernel_30252\3881968528.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM policies LIMIT 10;", post_con)


,id,reportedzipcode,propertystate,reportedcity,countycode,latitude,longitude,policyeffectivedate,policyterminationdate,propertypurchasedate,...,construction,numberoffloorsininsuredbuilding,lowestfloorelevation,elevatedbuildingindicator,floodproofedindicator,postfirmconstructionindicator,policytermindicator,ratemethod,programtypeindicator,nfipratedcommunitynumber
0,8c56361f-c29d-4321-a203-219b3b3af875,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-05-16,2022-05-16,2021-04-16,...,False,2,None,False,False,True,1,7,False,485500
1,3cfae310-e162-412d-818d-7744d2761c14,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-06-15,2022-06-15,2018-01-01,...,False,1,None,False,False,True,1,7,False,485500
2,78470d8d-aefc-43ac-9ab5-477a8225df1e,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-06-22,2022-06-22,None,...,False,2,None,False,False,True,1,7,False,485500
3,0cf829af-c9c6-4068-b373-b5fb36593173,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-08-16,2022-08-16,1992-04-01,...,False,1,None,False,False,True,1,7,False,485500
4,cadb086f-3f85-4044-b7cb-0484ab74c676,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-05-07,2022-05-07,2021-05-07,...,False,1,None,False,False,False,1,7,False,485500
5,207b74fd-088d-4d5f-b73d-b18d600a720b,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-09-02,2022-09-02,None,...,False,1,None,False,False,False,1,7,False,485500
6,f979b37d-863c-4872-aa4e-94bb603f6f59,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-07-23,2022-07-23,2014-07-01,...,False,1,None,False,False,True,1,7,False,485500
7,11f3cda4-b9c5-4227-abc4-74cc13864f33,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-06-05,2022-06-05,2001-01-01,...,False,1,None,False,False,True,1,7,False,485500
8,00bb90ba-5ba9-4dd7-b12a-cae842d387a5,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-07-20,2022-07-20,2015-04-15,...,False,1,None,False,False,True,1,7,False,485500
9,76024b5c-ad0e-4700-ba2c-8c04072b2868,77651,TX,Currently Unavailable,48245,30.0,-94.0,2021-05-30,2022-05-30,None,...,False,1,None,False,False,True,1,7,False,485500


In [50]:
# Now that relational database is ready, it is crucial to think about optimizing the performance as it is a huge dataset
# Optimization practice - Index the columns which will be frequently used for querying during analysis

def create_idx(attr):
    query  = f"CREATE INDEX IF NOT EXISTS idx_{attr} ON policies ({attr});"
    return query

idx_attr = ["reportedZipCode", "propertyState", "reportedCity", "policyEffectiveDate", "policyTerminationDate", "totalInsurancePremiumOfThePolicy"]
for attr in idx_attr:
    cur.execute(create_idx(attr))